In [13]:
import numpy as np
import os
import pandas as pd

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker .config import ExportFormat
from tflite_model_maker .text_classifier import AverageWordVecSpec
from tflite_model_maker .text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [14]:
import tensorflow as tf

print(len(tf.config.list_physical_devices('GPU')))

0


In [15]:
df = pd.read_csv('emotion_dataset.csv')

label_map = {0: 'neutral', 1: 'joy', 2: 'sadness', 3: 'angry', 4:'fear'}

df = df.loc[ (df['Emotion'] == 'sadness') | (df['Emotion'] =='fear') | (df['Emotion'] == 'anger')| (df['Emotion'] == 'joy')| (df['Emotion'] == 'neutral')]
print(df['Emotion'].value_counts())
#df=df.drop(df.query("Emotion == 'joy'").sample(frac = 0.5).index)
print(df['Emotion'].value_counts())
df=df.drop(['Text', 'Unnamed: 0'], axis = 1)
df['Clean_Text'] = df['Clean_Text'].str.lower()
df.head()



joy        11045
sadness     6722
fear        5410
anger       4297
neutral     2254
Name: Emotion, dtype: int64
joy        11045
sadness     6722
fear        5410
anger       4297
neutral     2254
Name: Emotion, dtype: int64


,Emotion,Clean_Text
0,neutral,NaN
1,joy,sage act upgrade list tommorow
2,sadness,way homegirl baby funeral man hate funerals sh...
3,joy,eye true hazel eyeand brilliant regular feat...
4,joy,ugh babe hugggzzz u babe naamazed nga ako e...


In [16]:
train = df.sample(frac = 0.8, random_state = 42)
test = df.loc[~df.index.isin(train.index)]

In [17]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [18]:
spec = model_spec.get('average_word_vec')
train_data = DataLoader.from_csv(filename = 'train.csv', text_column = 'Clean_Text', label_column = 'Emotion', model_spec = spec, is_training = True)
test_data = DataLoader.from_csv(filename = 'test.csv', text_column = 'Clean_Text', label_column = 'Emotion', model_spec = spec, is_training = False)


In [23]:
ds = train_data.gen_dataset()
count = 0;
for i in ds.take(10):
  print(i)

(<tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[   1,   36,  469, 7763, 7764,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  

In [19]:
import tflite_model_maker
tflite_model_maker.config.QuantizationConfig(inference_input_type=tf.float32, inference_output_type=tf.float32)

In [20]:
model = text_classifier.create(train_data, model_spec = spec, epochs = 80)

Epoch 1/80
743/743 [==============================] - 2s 2ms/step - loss: 1.5042 - accuracy: 0.3682
Epoch 2/80
743/743 [==============================] - 1s 1ms/step - loss: 1.4911 - accuracy: 0.3694
Epoch 3/80
743/743 [==============================] - 1s 1ms/step - loss: 1.4760 - accuracy: 0.3695
Epoch 4/80
743/743 [==============================] - 1s 1ms/step - loss: 1.4341 - accuracy: 0.3748
Epoch 5/80
743/743 [==============================] - 1s 1ms/step - loss: 1.3696 - accuracy: 0.4156
Epoch 6/80
743/743 [==============================] - 1s 1ms/step - loss: 1.3079 - accuracy: 0.4550
Epoch 7/80
743/743 [==============================] - 1s 1ms/step - loss: 1.2575 - accuracy: 0.4796
Epoch 8/80
743/743 [==============================] - 1s 1ms/step - loss: 1.2141 - accuracy: 0.5013
Epoch 9/80
743/743 [==============================] - 1s 1ms/step - loss: 1.1757 - accuracy: 0.5190
Epoch 10/80
743/743 [==============================] - 1s 1ms/step - loss: 1.1472 - accuracy: 0.5466

In [21]:
loss, acc = model.evaluate(test_data)

186/186 [==============================] - 0s 1ms/step - loss: 0.9103 - accuracy: 0.6867


In [22]:
model.export(export_dir='label/vocab', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])